<a href="https://colab.research.google.com/github/hoonputer/Section1_Project/blob/main/n224_discussion_3%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N224 Model Tuning**

## 오늘의 목표
- 하이퍼파라미터 튜닝으로 모델의 성능을 향상시킬 수 있습니다.
- Grid/Random/Bayesian search CV를 사용할 수 있습니다.
- Grid/Random/Bayesian search CV의 차이점과 어떤 경우에 어떤 방법을 사용하면 좋을지 설명할 수 있습니다.


## **개념 Topic**
> 오늘은 하이퍼파라미터 튜닝에 대해 배웠습니다.

- grid search, randomized search, bayesian search의 차이점을 생각하면서 어떤 경우에 어떤 방법을 사용하면 좋을지 논의해 보세요.

- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

Grid search :   랜덤사용이후, grid를 사용  
Randomized search :   모델링시에, 하나 뽑은후 범위를 넓히고    
Bayesian search :   grid 이후  , 


### **코치 comment**
- Grid: 시간적인 여유가 충분하고 경험적으로 해당 데이터에서 하이퍼파라미터가 어떠한 범위를 가질때 성능이 높다는 지식이 있을 경우. 정확한 탐색을 위해서 사용.
- Random: 하이퍼파라미터 범위에 대한 감이 없어서 넓은 범위로 최적의 조합을 찾으려고 하는 경우. Grid 보다 효율적으로 탐색하고자 할 때 사용.
- Bayesian: 한정된 자원 내에 Grid, Random보다 최적의 조합을 효율적으로 탐색하고자 하는 경우.
- 언제나 무조건적으로 좋은 튜닝 방법은 없기 때문에 다양한 시도를 해보고 가장 좋은 조합을 선택하는 것이 좋습니다.

scikit-learn에서 hyperopt를 구현한 라이브러리도 있습니다.
[hpsklearn 가이드 참고문서](https://hyperopt.github.io/hyperopt-sklearn/)

## **코딩 Topic**


In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install xgboost==1.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.4
    Uninstalling xgboost-1.7.4:
      Successfully uninstalled xgboost-1.7.4


### **Part.1 : 데이터 준비**
지난 노트에서 사용했던 [통신사 고객 이탈 여부 데이터](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)를 사용합니다.

> **Data Description**

- customerID : Customer ID
- gender : Whether the customer is a male or a female
- SeniorCitizen : Whether the customer is a senior citizen or not (1, 0)
- Partner : Whether the customer has a partner or not (Yes, No)
- Dependents : Whether the customer has dependents or not (Yes, No)
- tenure : Number of months the customer has stayed with the company
- PhoneService : Whether the customer has a phone service or not (Yes, No)
- MultipleLines : Whether the customer has multiple lines or not (Yes, No, No phone service)
- InternetService : Customer’s internet service provider (DSL, Fiber optic, No)
- OnlineSecurity : Whether the customer has online security or not (Yes, No, No internet service)
- OnlineBackup : Whether the customer has online backup or not (Yes, No, No internet service)
- DeviceProtection : Whether the customer has device protection or not (Yes, No, No internet service)
- TechSupport : Whether the customer has tech support or not (Yes, No, No internet service)
- StreamingTV : Whether the customer has streaming TV or not (Yes, No, No internet service)
- StreamingMovies : Whether the customer has streaming movies or not (Yes, No, No internet service)
- Contract : The contract term of the customer (Month-to-month, One year, Two year)
- PaperlessBilling : Whether the customer has paperless billing or not (Yes, No)
- PaymentMethod : The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- MonthlyCharges : The amount charged to the customer monthly
- TotalCharges : The total amount charged to the customer
- Churn : Whether the customer churned or not (Yes or No)

#### **1-1. 데이터 불러오기 및 전처리**
- 데이터셋을 불러오고 지난 노트에서 수행한 전처리 + 추가로 시도해보고 싶은 전처리를 수행해보세요.

In [ ]:
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/renewal/mldl/Customer_Churn.csv')
display(df.head())
print(df.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


(7043, 21)


In [ ]:
import numpy as np

# customerID 열 제거
df.drop('customerID', axis=1, inplace=True)

# 결측치 및 중복 샘플 제거
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

# TotalCharges에서 공란인 데이터를 0으로 변환 후, 데이터 타입을 float로 변환
df['TotalCharges'] = df['TotalCharges'].replace(' ', '0').astype(float)

#### **1-2. 데이터셋 분리**
- 특성과 타겟을 분리하세요.
  - 타겟은 `Churn` 입니다.
  - 타겟 데이터를 `No`는 0으로, `Yes`는 1로 변환하세요.
- 데이터를 train/test set으로 분리하세요.
  - 적절한 비율로 train/test set을 나누세요.
  - `random_state`로 시드를 고정하세요.
  - 타겟 클래스의 비율이 유지되도록 분리하세요.

In [ ]:
from sklearn.model_selection import train_test_split

# 타겟 지정, 타겟 데이터 (No == 0 , Yes == 1) 로 변경
target = 'Churn'
df[target] = df[target].replace(regex={'No' : 0, 'Yes' : 1})

# 훈련, 검증, 테스트 셋 분리
X = df.drop(columns=target)
y = df[target]

# 비율 = 8:2
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.2, stratify=y_train, random_state=42)

### **Part.2 : 모델 학습 및 예측**

#### **2-1. 기준모델 설정**
- 데이터가 불균형하기 때문에 F1-score를 평가지표로 사용합니다.

  1) 타겟의 최빈값으로 기준 모델을 설정하면 F1-score가 0이 나오기 때문에 Decision Tree 모델을 기준모델로 사용합니다.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - 기준모델의 하이퍼파라미터를 자유롭게 설정하세요.

  2) `cross_val_score` 함수를 사용해서 기준모델의 검증 성능을 확인해보세요. 

  3) 검증 성능을 확인한 후 모델을 학습시켜서 학습 데이터에서의 성능을 확인해보세요. 
    - 과적합이 발생했다면 특정 하이퍼파라미터를 직접 조정하여 해소해보세요.
      - 하이퍼파라미터가 달라졌다면 다시 검증 성능을 확인한 후 학습 성능을 확인해야합니다.
    - 과적합을 해소한 기준모델의 검증 F1-score는 어떻게 되나요?
      - 이 검증 F1-score를 기준모델의 성능으로 사용합니다.

In [ ]:
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score
from xgboost import XGBClassifier

In [ ]:
# 결정트리모델 파이프라인으로 생성
pipe_dt = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    DecisionTreeClassifier(random_state=42, max_depth=4)
)

```
max_depth: None = None  
min_samples_split: int = 2   
min_samples_leaf: int = 1   
min_weight_fraction_leaf: float = 0   
max_features: None = None   
random_state: None = None   
max_leaf_nodes: None = None   
min_impurity_decrease: float = 0   
class_weight: None = None,   
ccp_alpha: float = 0
```

In [ ]:
# 결정트리 기준모델 지정
base = pipe_dt.fit(X_train, y_train)

# cross_val_score 함수 사용
cva_results = cross_val_score(
    base,
    X_train,
    y_train,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

print('F1-score for each fold : ', np.round(cva_results, 2))
print('average F1 for model : ', np.mean(cva_results).round(2))
print('std of result : ', np.std(cva_results).round(2))

F1-score for each fold :  [0.51 0.55 0.55 0.56 0.54]
average F1 for model :  0.54
std of result :  0.02


In [ ]:
print('train 정확도 : ', base.score(X_train, y_train))
print('val 정확도 : ', base.score(X_val, y_val))

train 정확도 :  0.7960819234194123
val 정확도 :  0.8069395017793595


초기  
train 정확도 :  0.9977738201246661  
val 정확도 :  0.7188612099644128  
F1-score for each fold :  [0.43 0.5  0.49 0.51 0.5 ]  
average F1 for model :  0.49  
std of result :  0.03  

max_depth =4  
F1-score for each fold :  [0.51 0.55 0.55 0.56 0.54]  
average F1 for model :  0.54  
std of result :  0.02  
train 정확도 :  0.7960819234194123  
val 정확도 :  0.8069395017793595  

max_depth =6  
F1-score for each fold :  [0.55 0.56 0.49 0.58 0.53]  
average F1 for model :  0.54  
std of result :  0.03  
train 정확도 :  0.8130008904719501  
val 정확도 :  0.800711743772242  

#### **2-2. 하이퍼파라미터 튜닝**
- Grid, Random, Bayesian Search CV 중 2가지 방법을 사용해서 하이퍼파라미터 튜닝을 진행해보세요.
- 추천 튜닝 방법
  - 튜닝1) Grid/Random Search CV 중 택1
  - 튜닝2) Bayesian Search CV



##### **GridSearch CV**
- GridSearch CV를 활용해서 위의 기준모델보다 성능이 더 좋은 모델을 만들어보세요.
  - 튜닝 시, RandomForest와 XGBoost 모델 중 하나를 사용하세요.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - `random_state`로 모델의 시드를 고정하세요.
  - 튜닝 시, 하이퍼파라미터는 최소 2개, 최대 4개까지 사용하세요.
    - 시간 관계상 하이퍼파라미터 튜닝의 범위가 너무 넓지 않게 설정하세요. (튜닝 시간 3분 이내)
    - cv는 최대 3까지만 설정하세요.
    - 하이퍼파라미터의 조합과 범위를 아래와 같이 설정한 이유는 무엇인가요?
  - 최적의 하이퍼파라미터 조합을 가진 모델로 학습/검증 F1-score를 출력해보세요.
    - 검증 F1-score: `그리드서치.best_score_`
    - 최적의 모델: `그리드서치.best_estimator_`
    - [scikit-learn Gridsearch 공식문서](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
    - 기준모델보다 성능이 좋은 모델과 하이퍼파라미터 조합을 발견했나요?
      - score와 조합을 출력해보세요.

In [ ]:
# 모델 생성

pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy="median"),
    # RandomForestClassifier(),
    # XGBClassifier(),
    ),
)
print(pipe)

In [ ]:
# 파라미터 지정
params = {
    "xgbclassifier__max_depth": [2, 4, 6],
    "xgbclassifier__min_child_weight": [2, 4, 8],
    "xgbclassifier__colsample_bytree": [0.6, 0.8, 1.0],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipe, param_grid=params, scoring="roc_auc", cv=3, verbose=3)
grid_search.fit(X_train, y_train)

In [ ]:
print("최적 하이퍼파라미터: ", grid_search.best_params_)
print("최적 AUC: ", grid_search.best_score_)

##### **RandomSearch CV**
- RandomSearch CV를 활용해서 위의 기준모델보다 성능이 더 좋은 모델을 만들어보세요.
  - 튜닝 시, RandomForest와 XGBoost 모델 중 하나를 사용하세요.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - `random_state`로 모델의 시드를 고정하세요.
  - 튜닝 시, 하이퍼파라미터는 최소 2개, 최대 4개까지 사용하세요.
    - 시간 관계상 하이퍼파라미터 튜닝의 범위가 너무 넓지 않게 설정하세요. (튜닝 시간 3분 이내)
    - n_iter는 최대 10, cv는 최대 3까지만 설정하세요.
    - `random_state`로 RandomSearch CV의 시드를 고정하세요.
    - 하이퍼파라미터의 조합과 범위를 아래와 같이 설정한 이유는 무엇인가요?
  - 최적의 하이퍼파라미터 조합을 가진 모델로 학습/검증 F1-score를 출력해보세요.
    - 검증 F1-score: `랜덤서치.best_score_`
    - 최적의 모델: `랜덤서치.best_estimator_`
    - [RandomSearch 공식문서](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
    - 기준모델보다 성능이 좋은 모델과 하이퍼파라미터 조합을 발견했나요?
      - score와 조합을 출력해보세요.

In [ ]:
# 모델 생성

pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy="mean"),
    XGBClassifier(max_depth=2, xgbclassifier__colsample_bytree= 0.5909124836035503, min_child_weight=2),
    )

In [ ]:
from scipy.stats.distributions import uniform

params = {
    "simpleimputer__strategy": ["median", "mean"],
    "xgbclassifier__max_depth": [2, 4, 6],
    "xgbclassifier__min_child_weight": [2, 4, 8],
    "xgbclassifier__colsample_bytree": uniform(
        loc=0.5, scale=0.5
    ),  # 0.5 ~ 1 사이의 uniform 분포로 범위를 지정해 줍니다.
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

randomized_search = RandomizedSearchCV(
    pipe,
    param_distributions=params,
    scoring="f1",     # 스코어링은 따로 외부에서 함수를 지정하여 사용. 한 모델에 2개는 안됨!
    n_iter=10,
    cv=3,
    verbose=3,
    random_state=42,
)
randomized_search.fit(X_train, y_train)

In [ ]:
print("최적 하이퍼파라미터: ", randomized_search.best_params_)
print("최적 F1-scored: ", randomized_search.best_score_)

최적 하이퍼파라미터:  {'simpleimputer__strategy': 'mean', 'xgbclassifier__colsample_bytree': 0.5909124836035503, 'xgbclassifier__max_depth': 2, 'xgbclassifier__min_child_weight': 2}
최적 F1-scored:  0.578836521006049


In [ ]:
simpleimputer__strategy': 'mean', 
'xgbclassifier__colsample_bytree': 0.5909124836035503, 
'xgbclassifier__max_depth': 2, 
'xgbclassifier__min_child_weight': 2

SyntaxError: ignored

##### **BayesianSearch CV**
- BayesianSearch CV를 활용해서 위의 기준모델보다 성능이 더 좋은 모델을 만들어보세요.
  - 튜닝 시, RandomForest와 XGBoost 모델 중 하나를 사용하세요.
    - 범주형 데이터는 적절한 encoding을 사용하여 변환하세요.
    - `random_state`로 모델의 시드를 고정하세요.
  - 튜닝 시, 하이퍼파라미터는 최소 2개, 최대 4개까지 사용하세요.
    - 시간 관계상 하이퍼파라미터 튜닝의 범위가 너무 넓지 않게 설정하세요. (튜닝 시간 3분 이내)
    - cross_val_score
의 cv는 3 이하, fmin의 max_evals는 10 이하로 설정하세요.
    - rstate로 시드를 고정하세요.
      - `rstate = np.random.default_rng(...)`
    - 하이퍼파라미터의 조합과 범위를 아래와 같이 설정한 이유는 무엇인가요?

- BayesianSearch CV는 `best_estimator_` 메소드를 제공하고 있지 않아 최적의 조합을 찾았다면 해당 조합으로 모델을 재학습 시켜주어야합니다.
  - 발견한 하이퍼파라미터 조합을 사용해서 모델을 재학습 시키세요.
  - 재학습된 최종 모델의 학습/검증 F1-score를 출력해보세요.
  - 검증 F1-score: `-베이지안서치.best_trial["result"]["loss"]`
  - 기준모델보다 성능이 좋은 모델과 하이퍼파라미터 조합을 발견했나요?
      - score와 조합을 출력해보세요.

In [ ]:
from hyperopt import hp

params = {
    "simpleimputer__strategy": hp.choice("strategy", ["median", "mean"]),
    "xgbclassifier__max_depth": hp.quniform("max_depth", 2, 10, 2),
    "xgbclassifier__min_child_weight": hp.quniform("min_child_weight", 2, 10, 2),
    "xgbclassifier__colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
}

In [ ]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score

def get_pipe(params):
    params["xgbclassifier__max_depth"] = int(
        params["xgbclassifier__max_depth"]
    )  # max_depth는 정수형으로 변환해 줍니다.
    pipe = make_pipeline(
        OrdinalEncoder(),
        SimpleImputer(strategy="median"),
        XGBClassifier(
            objective="binary:logistic",
            eval_metric="error",
            n_estimators=200,
            random_state=42,
            n_jobs=-1,
            learning_rate=0.1,
            use_label_encoder=False,
        ),
    )
    pipe = pipe.set_params(**params)
    return pipe


def fit_and_eval(params):
    pipe = get_pipe(params)  # 주어진 params로 파이프라인을 만들어 가져옵니다.
    score = cross_val_score(pipe, X_train, y_train, cv=3, scoring="roc_auc")
    avg_cv_score = np.mean(score)
    # roc_auc는 클수록 좋은 metric이므로, hyperopt.fmin이 roc_auc를 최대화하도록 하기 위해 -부호를 붙여 반환합니다.
    return {"loss": -avg_cv_score, "status": STATUS_OK}


trials = (
    Trials()
)  # Trials() 객체를 fmin에 함께 넣어 실행하면, 실행되는 매 fit의 학습 정보 및 학습 결과가 해당 객체에 모두 저장됩니다.

best_params = fmin(
    fn=fit_and_eval, trials=trials, space=params, algo=tpe.suggest, max_evals=10,
    rstate = np.random.default_rng(2023)
)  # max_evals 횟수만큼 하이퍼파라미터 조합을 탐색합니다.

In [ ]:
print("최적 하이퍼파라미터: ", trials.best_trial["misc"]["vals"])
print("최적 AUC: ", -trials.best_trial["result"]["loss"])

### **Part.3 : 모델 결과 해석**

#### **3-1. 최종 모델 선정 및 일반화 성능 평가**
- 사용한 튜닝 방법 중 가장 성능이 좋은 튜닝 방법은 무엇인가요?
  - 이 방법이 **항상** 다른 튜닝 방법에 비해 성능이 좋을까요?
  - 항상 좋다면 왜 좋은지, 항상 좋진 않다면 왜 항상은 아닌지 논의해보세요.
- 가장 성능이 좋은 튜닝 방법을 사용한 모델을 최종 모델로 선정하세요.
  - 최종 모델의 일반화 성능(test set에서의 F1-score)을 확인해보세요.
  - 과적합이 발생하진 않았나요?
    - 과적합이 발생했다면 어떻게 해소할 수 있을지 논의해보세요.

In [ ]:
y_pred = .predict(X_test)
print(classification_report(y_val, y_pred))

### **Conclusion**
> 오늘 Topic을 수행한 결과를 바탕으로, 다음 사항에 대해 답해 주세요.

1. 어떤 튜닝 방법들을 사용했나요?
2. 튜닝에 사용한 하이퍼파라미터 조합과 범위의 설정 이유는 무엇인가요?
3. 가장 좋은 튜닝 방법과 해당 모델의 학습/검증/테스트 성능은 어떻게 되나요?
4. 일반화 성능(test set 성능)을 올리기 위해서 어떤 방법을 사용해볼 수 있을까요?

- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

1. 
2. 
3. 
4. 



## **심화 Topic(Optional)**
- 위의 **2-2. 하이퍼파라미터 튜닝** 파트에서 사용하지 않은 나머지 튜닝 방법을 주어진 가이드에 따라 수행해보세요.